In [ ]:
import atlite
import xarray as xr
import rasterio as rio
import rioxarray as rioxr

import os.path
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
plt.style.use('dark_background')

from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer

import cartopy.crs as ccrs
from cartopy.crs import PlateCarree as plate
import cartopy.io.shapereader as shpreader

from osgeo import gdal

from exp_utils import wgp_functions

plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
# Get shape of Germany 

shpfilename = shpreader.natural_earth(resolution='10m',
                                          category='cultural',
                                          name='admin_0_countries')
    
reader = shpreader.Reader(shpfilename)

Germany = gpd.GeoSeries({r.attributes['NAME_EN']: r.geometry
                    for r in reader.records()},
                    crs='epsg:4326'
                   ).reindex(['Germany'])

## Basic handeling of altite.Cutout and xarray-DataSets

In [ ]:
# Read in cutout(s) 

src = []
root = 'C:/CaT/Masterthesis/repository/data/era5/Germany/'

for i in pd.date_range('2019-01-01' , '2021-12-31', freq = 'M').strftime('%Y-%m'): 
    
    src.append(f'{root}{i}.nc')            
               
cutout = wgp_functions.merge_cutouts(src)

cutout

In [ ]:
cutout.data

In [ ]:
# Restrict time dimension to one frame

fig, ax = plt.subplots()
cutout.data['wnd100m'].isel(time = 0).plot(ax=ax)
Germany.plot(ax=ax, facecolor = 'None', edgecolor = 'k', lw=1)
ax.set_title('Wind100m @ '+str(cutout.data.time.values[0])[0:16])
plt.show()

In [ ]:
# Mean Through Time

fig, ax = plt.subplots()
cutout.data['wnd100m'].mean(['time']).plot(ax=ax)
Germany.plot(ax=ax, facecolor = 'None', edgecolor = 'k', lw=1)
ax.set_title(f'Wind100m Mean {str(cutout.data.time.values[0])[0:10]} to {str(cutout.data.time.values[-1])[0:10]}')
plt.show()

In [ ]:
# Mean Through Time

fig, ax = plt.subplots(figsize=(16,8))
cutout.data['wnd100m'].mean(['x', 'y']).rolling(time=24*7).mean().plot(ax=ax)
ax.set_title(f'Wind100m Weekly-Mean for Germany{str(cutout.data.time.values[0])[0:10]} to {str(cutout.data.time.values[-1])[0:10]}')
plt.show()

In [ ]:
#Time slice via sel 
cutout.data.sel(time=slice('2020-01-02', '2020-01-03'))

In [ ]:
#Integer slice via isel
cutout.data.isel(x=slice(10, 20))

## Datasets with different resolutions: ERA5 & GWA

In [ ]:
# Import GWA Data for Germany
gwa = rioxr.open_rasterio('C:/CaT/Masterthesis/repository/data/gwa/DEU_wind-speed_100m.tif')

gwa = gwa.rename({'x' : 'lon', 'y' : 'lat'})

# Move the spatial_ref to attributes
gwa.attrs['spatial_ref'] = gwa['spatial_ref'].attrs
gwa = gwa.drop(['spatial_ref'])

# Drop the flat band dimension 
gwa = gwa.squeeze('band')
gwa = gwa.reset_coords('band', drop=True)
 
# Interpolate -> downscale GWA grid to ERA5 resolution
gwa_interpol = gwa.interp(lat=cutout.data.lat, lon=cutout.data.lon, method = 'linear')

In [ ]:
cutout_interpol = cutout.data.swap_dims({'y' : 'lat', 'x' : 'lon'})

#Interpolate -> Upscale ERA5 grid to GWA resolution. Take mean beforehand so ds is flat. for 2 months the resulting ds is too big
cutout_interpol = cutout_interpol.mean(['time']).interp(lat=gwa.lat, lon=gwa.lon, method = 'linear')

In [ ]:
fig = plt.figure(figsize=(12,12), dpi = 100)

gs = fig.add_gridspec(50, 3, width_ratios=[0.475, 0.475, 0.05])
ax = []
ax.append(fig.add_subplot(gs[0:22, 0]))
ax.append(fig.add_subplot(gs[0:22, 1]))
ax.append(fig.add_subplot(gs[27:49, 0]))
ax.append(fig.add_subplot(gs[27:49, 1]))
ax.append(fig.add_subplot(gs[11:39, 2]))

fig.suptitle("Wind Data, scaled and transformed", size=16)

cutout.data['wnd100m'].mean('time').plot(ax = ax[0], vmin = gwa.min(), vmax = gwa.max(), add_colorbar=False)
Germany.plot(ax=ax[0], facecolor = 'None', edgecolor = 'k', lw=1)
ax[0].set_title(f'ERA5 Original Data {str(cutout.data.time.values[0])[0:10]} to {str(cutout.data.time.values[-1])[0:10]}')
ax[0].set_ylabel("lat")
ax[0].set_xlabel("lon")

gwa.plot(ax=ax[1], vmin = gwa.min(), vmax = gwa.max(), add_colorbar=False)
Germany.plot(ax=ax[1], facecolor = 'None', edgecolor = 'k', lw=1)
ax[1].set_ylim(cutout.data.lat.min(), cutout.data.lat.max())
ax[1].set_xlim(cutout.data.lon.min(), cutout.data.lon.max())
ax[1].set_title('GWA [Original Data]')

cutout_interpol['wnd100m'].plot(ax=ax[2], vmin = gwa.min(), vmax = gwa.max(), add_colorbar=False)
Germany.plot(ax=ax[2], facecolor = 'None', edgecolor = 'k', lw=1)
ax[2].set_title('ERA5 [Upscaled to GWA Grid]')
ax[2].set_ylim(cutout.data.lat.min(), cutout.data.lat.max())
ax[2].set_xlim(cutout.data.lon.min(), cutout.data.lon.max())

gwa_interpol.plot(ax=ax[3], vmin = gwa.min(), vmax = gwa.max(), add_colorbar=False)
Germany.plot(ax=ax[3], facecolor = 'None', edgecolor = 'k', lw=1)
ax[3].set_title('GWA [Downscaled to ERA5 Grid]')
ax[3].set_ylabel("lat")
ax[3].set_xlabel("lon")

# Colorbar
norm = plt.Normalize(gwa.min(), gwa.max())
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])
plt.colorbar(sm, cax=ax[4], orientation='vertical')
ax[4].get_yaxis().labelpad = 10
ax[4].set_ylabel('meters/second', rotation = 270, fontsize = 10)

fig.subplots_adjust(top=0.925)

plt.savefig('C:/CaT/Masterthesis/repository/presentations/scaling.png', bbox_inches='tight', dpi = 100)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16,8), dpi = 100)

#mi, ma = (cutout_xr_wind_interpol/gwa).min().values, (cutout_xr_wind_interpol/gwa).max().values

from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

fig.suptitle(f'ERA5 Wind100m Mean {str(cutout.data.time.values[0])[0:10]} to {str(cutout.data.time.values[-1])[0:10]} / GWA Ratio Example', size=16)         
             
(cutout.data['wnd100m'].mean('time')/gwa_interpol).plot(ax=ax[0], vmin = 0, vmax = 2, cmap = cmap)
Germany.plot(ax=ax[0], facecolor = 'None', edgecolor = 'k', lw=1)
ax[0].set_ylim(cutout.data.y.min(), cutout.data.y.max())
ax[0].set_xlim(cutout.data.x.min(), cutout.data.x.max())
ax[0].set_title('Ratio(ERA5 / GWA [Downscaled])')
ax[0].set_ylabel("lat")
ax[0].set_xlabel("lon")

(cutout_interpol['wnd100m']/gwa).plot(ax=ax[1], vmin = 0, vmax = 2, cmap = cmap)
Germany.plot(ax=ax[1], facecolor = 'None', edgecolor = 'k', lw=1)
ax[1].set_title('Ratio(ERA5 [Upscaled] / GWA)')
ax[1].set_ylim(cutout.data.y.min(), cutout.data.y.max())
ax[1].set_xlim(cutout.data.x.min(), cutout.data.x.max())
ax[1].set_ylabel("lat")
ax[1].set_xlabel("lon")

plt.savefig('C:/CaT/Masterthesis/repository/presentations/ratio.png', bbox_inches='tight', dpi = 100)

plt.show()

In [ ]:
print('cutout / gwa downscaled: ' ,'\n',
      'mean: ',(cutout.data['wnd100m'].mean('time')/gwa_interpol).mean(skipna=True).values,'\n',
      'std: ',(cutout.data['wnd100m'].mean('time')/gwa_interpol).std(skipna=True).values,'\n',
      'min: ',(cutout.data['wnd100m'].mean('time')/gwa_interpol).min(skipna=True).values,'\n',
      'q10: ',(cutout.data['wnd100m'].mean('time')/gwa_interpol).quantile(q=0.1, skipna=True).values,'\n',
      'q50: ',(cutout.data['wnd100m'].mean('time')/gwa_interpol).quantile(q=0.5, skipna=True).values,'\n',
      'q90: ',(cutout.data['wnd100m'].mean('time')/gwa_interpol).quantile(q=0.9, skipna=True).values,'\n',
      'max: ', (cutout.data['wnd100m'].mean('time')/gwa_interpol).max(skipna=True).values
     )

In [ ]:
(cutout.data['wnd100m'].mean('time')/gwa_interpol).quantile(q=0.5, skipna=True).values

In [ ]:
print('cutout upscaled / gwa: ' ,'\n',
      'mean: ',(cutout_interpol['wnd100m']/gwa).mean(skipna=True).values,'\n',
      'std: ',(cutout_interpol['wnd100m']/gwa).std(skipna=True).values,'\n',
      'min: ',(cutout_interpol['wnd100m']/gwa).min(skipna=True).values,'\n',
      'q10: ',(cutout_interpol['wnd100m']/gwa).quantile(q=0.1, skipna=True).values,'\n',
      'q50: ',(cutout_interpol['wnd100m']/gwa).quantile(q=0.5, skipna=True).values,'\n',
      'q90: ',(cutout_interpol['wnd100m']/gwa).quantile(q=0.9, skipna=True).values,'\n',
      'max: ', (cutout_interpol['wnd100m']/gwa).max(skipna=True).values
     )




## Build correction factor vgl. Ryberg et al 2018

# ${v(t)} = {v}_{MERRA}{(t)} * \frac{{v}_{GWA, 50m}}{{v}_{MERRA, avg}}$

In [ ]:
cutout_corrected = cutout.data['wnd100m'] * (gwa_interpol / cutout.data['wnd100m'].mean('time'))

In [ ]:
(gwa_interpol / cutout.data['wnd100m'].mean('time')).mean().values

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16,8), dpi = 100)

#mi, ma = (cutout_xr_wind_interpol/gwa).min().values, (cutout_xr_wind_interpol/gwa).max().values

fig.suptitle("ERA5 Wind100m 2020-01-01 00:00, Long-Run Mean Corrected with GWA", size=16)         
                 

cutout.data['wnd100m'].isel(time = 0).plot(ax=ax[0], vmin = 0, vmax = 12)
Germany.plot(ax=ax[0], facecolor = 'None', edgecolor = 'k', lw=1)
ax[0].set_title('Raw')
ax[0].set_ylim(cutout.data.y.min(), cutout.data.y.max())
ax[0].set_xlim(cutout.data.x.min(), cutout.data.x.max())
ax[0].set_ylabel("lat")
ax[0].set_xlabel("lon")
      
    
cutout_corrected.isel(time = 0).plot(ax=ax[1], vmin = 0, vmax = 12)
Germany.plot(ax=ax[1], facecolor = 'None', edgecolor = 'k', lw=1)
ax[1].set_ylim(cutout.data.y.min(), cutout.data.y.max())
ax[1].set_xlim(cutout.data.x.min(), cutout.data.x.max())
ax[1].set_title('Corrected')
ax[1].set_ylabel("lat")
ax[1].set_xlabel("lon")


plt.savefig('C:/CaT/Masterthesis/repository/presentations/correctedhour.png', bbox_inches='tight', dpi = 100)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16,8), dpi = 100)

#mi, ma = (cutout_xr_wind_interpol/gwa).min().values, (cutout_xr_wind_interpol/gwa).max().values

fig.suptitle(f'ERA5 Wind100m {str(cutout.data.time.values[0])[0:10]} to {str(cutout.data.time.values[-1])[0:10]}, Long-Run Mean Corrected with GWA' , size=16)                         

cutout.data['wnd100m'].mean('time').plot(ax=ax[0], vmin = 0, vmax = 12)
Germany.plot(ax=ax[0], facecolor = 'None', edgecolor = 'k', lw=1)
ax[0].set_title('Raw')
ax[0].set_ylim(cutout.data.y.min(), cutout.data.y.max())
ax[0].set_xlim(cutout.data.x.min(), cutout.data.x.max())
ax[0].set_ylabel("lat")
ax[0].set_xlabel("lon")
      
    
cutout_corrected.mean('time').plot(ax=ax[1], vmin = 0, vmax = 12)
Germany.plot(ax=ax[1], facecolor = 'None', edgecolor = 'k', lw=1)
ax[1].set_ylim(cutout.data.y.min(), cutout.data.y.max())
ax[1].set_xlim(cutout.data.x.min(), cutout.data.x.max())
ax[1].set_title('Corrected')
ax[1].set_ylabel("lat")
ax[1].set_xlabel("lon")


plt.savefig('C:/CaT/Masterthesis/repository/presentations/correctedmonths.png', bbox_inches='tight', dpi = 100)

plt.show()

In [ ]:
cutout.data